In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from numpy import load
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.0 MB/s eta 0:00:00


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/ILDC_expert/anno_dataset.csv')

In [4]:
import pickle
file_path = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation_caselawbert/only_annotation/attention_anno/att_wt_of_caselawbert_bigru_anno.pkl'
with open(file_path, 'rb') as file:
    att_scores_anno = pickle.load(file)

In [5]:
output_dir = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation_caselawbert/CaseLawBERT_bigru_occ/saved_model_multi_caselawbert'
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [6]:
def caselawbert_detok(tokens):
    # Initialize an empty list for the detokenized sequence
    detokenized_sequence = []

    # Initialize a variable for handling '##' signs
    current_token = ""

    # Iterate through the tokens in the sequence
    for token in tokens:
        if token.startswith("##"):
            # If the token starts with '##', remove '##' and append the rest of the token to the previous token
            current_token += token[2:]
        else:
            # If the token does not start with '##', check if it is a punctuation mark
            if current_token and token in string.punctuation:
                # If the current token is a punctuation mark, attach it to the previous token
                current_token += token
            else:
                # If the current token is not a punctuation mark, add the previous token (if present) to the detokenized sequence
                if current_token:
                    detokenized_sequence.append(current_token)
                # Add the current token to the detokenized sequence
                current_token = token

    # Make sure to add the last token to the detokenized sequence
    if current_token:
        detokenized_sequence.append(current_token)


    tokens_concat = []

    prev_tok=""

    for token in detokenized_sequence:
      if token.endswith('-'):
          prev_tok = token
      else:
          if (prev_tok != ""):
              token = prev_tok + token
              prev_tok = ""
          tokens_concat.append(token)

    if (prev_tok!=""):
        tokens_concat.append(prev_tok)

    # Join the tokens in the detokenized sequence into a string
    detokenized_text = " ".join(tokens_concat)

    return detokenized_text

In [7]:
max_indexes = []
for i in range(len(att_scores_anno)):
  index_max= att_scores_anno[i][0].argmax()
  max_indexes.append(index_max)

In [8]:
final_explanations_att = {}
for i in range(len(df)):
  all_toks = tokenizer.tokenize(df.iloc[i]['text'])
  if(len(all_toks) > 10000):
      all_toks = all_toks[len(all_toks)-10000:]
  splitted_toks = []
  l=0
  r=510
  while(l<len(all_toks)):
    splitted_toks.append(all_toks[l:min(r,len(all_toks))])
    l+=410
    r+=410

  c= splitted_toks[max_indexes[i]]
  new_c = ["." if token == "[UNK]" else token for token in c]
  explanation_of_this_doc = caselawbert_detok(new_c)
  doc_name = df.iloc[i]['name']
  final_explanations_att[doc_name] = explanation_of_this_doc

In [9]:
final_explanations_att

{'1960_12.txt': 'that of the tribunal and it reversed the findings of the tribunal and answered the question in a manner suggestive of an appellate rather than advisory jurisdiction.',
 '1953_14.txt': 'is certainly less than the right which was available before. a provision which is calculated to deprive the appellant of the unfettered right of appeal cannot be regarded as a mere alteration in procedure. indeed the new requirement cannot be said merely to regulate the exercise of the appellants pre. existing right but in truth whittles down the right itself and cannot be regarded as a mere rule of procedure. finally. sri ganapathy lyer faintly urges that until actual assessment there can be no lis and. therefore. no right of appeal can accrue before that event. there are two answers to this plea. whenever there is a proposition by one party and an opposition to that proposition by another a lis arises. it may be conceded. though not deciding it. that when the assessee files his return 

In [11]:
import json
json_file = open("/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation_caselawbert/only_annotation/attention_anno/att_exp_anno.json", "w")
json.dump(final_explanations_att, json_file)